In [1]:
# library use
import re
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
from datetime import datetime
from datetime import timedelta
import datetime as dt
warnings.filterwarnings('ignore')
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import pickle
%config InlineBackend.figure_format = 'retina'
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [3]:
df = pd.read_csv('original_full_data.csv', index_col = 0)

In [4]:
mmdf = pd.read_csv('bloomberg_map_index_20200130.csv', index_col = 0)

In [5]:
df.head()

,date,GDPCQOQIndex,GDPCYOYIndex,GDPCTOT%Index,GPDITOC%Index,GPGSTOC%Index,RGCDCIPIIndex,GDPCUR$Index,GDPCURYIndex,GDPPIQQIndex,GDPCPCECIndex,CPIYOYIndex,CPIXYOYIndex,PCEDEFYIndex,PCECYOYIndex,CPICHNGIndex,CPUPXCHGIndex,FDIDFDMOIndex,FDIDSGMOIndex,FDIUFDYOIndex,FDIUSGYOIndex,USURTOTIndex,USUDMAERIndex,INJCJCIndex,INJCSPIndex,NFPTCHIndex,NFPPCHIndex,USMMMNCHIndex,AWHTOTLIndex,USWHTOTIndex,USEMTOTIndex,USEMNCHGIndex,USERTOTIndex,EMDINP1MIndex,JOLTOPENIndex,JOLTHIRSIndex,JOLTSEPSIndex,CHALYOY%Index,ADPCHNGIndex,USHEYOYIndex,USHETOT%Index,ECIYOYIndex,COSTNFR%Index,PRODNFR%Index,IPYOYIndex,IPCHNGIndex,CPTICHNGIndex,TMNOCHNGIndex,DGNOYOYIndex,DGNOCHNGIndex,MTIBCHNGIndex,MWINCHNGIndex,MGT2TBIndex,MGT2MAIndex,MGT2WHIndex,MGT2WHDUIndex,MGT2REIndex,IPIndex,MTSLRL$Index,AGWHTOTLIndex,NFPTIndex,PIDSPINXIndex,NAPMPMIIndex,NAPMNMIIndex,NAPMPRICIndex,MPMIUSCAIndex,MPMIUSMAIndex,MPMIUSSAIndex,CFNAIIndex,CFNAIMA3Index,OUTFGAFIndex,EMPRGBCIIndex,RCHSINDXIndex,CHPMINDXIndex,MAPMINDXIndex,OEUSKLARIndex,OEUSKLAPIndex,LEITOTLIndex,LEIYOYIndex,LEICHNGIndex,LEIAVGWIndex,LEIWKIJIndex,LEINWCNIndex,NAPMNEWOIndex,LEIMNOIndex,LEIBPIndex,LEISTKPIndex,LEILCIIndex,LEIIRTEIndex,LEIACEIndex,RSTAMOMIndex,RSTAXMOMIndex,RSTAYOYIndex,RSTAXYOYIndex,MBRXYOYWIndex,SAARTOTLIndex,SAARDTOTIndex,CONCCONFIndex,CONSSENTIndex,COMFCOMFIndex,COMFBTWRIndex,PITLYOYIndex,PITLCHNGIndex,PCECHY%Index,PCECHNCIndex,PIDSDPSIndex,CICRTOTIndex,EHCAUSIndex,USCABALIndex,USTBTOTIndex,USTBEXPIndex,USTBIMPIndex,USTBEXPYIndex,USTBIMPYIndex,IMP1YOY%Index,IMP1CHNGIndex,FRNTTOTLIndex,FRNTTNETIndex,FDTRIndex,USGG3MIndex,USGG10YRIndex,USTW$Index,JNVNYOYSIndex,JNVNIYOYIndex,MPMIJPCAIndex,MPMIJPMAIndex,MPMIJPSAIndex,CNGDPYOYIndex,GDPNTTLYIndex,CHEFTYOYIndex,CNPPIYIndex,CPMINDXIndex,CPMINORDIndex,CPMINEXOIndex,CNCILIIndex,CNFREXPYIndex,CNFRIMPYIndex,KOGDPYOYIndex,KOGDPQOQIndex,KOECPRCYIndex,KOECGCPYIndex,KOECGVTYIndex,KOECEXPYIndex,KOECIMPYIndex,KOECSPRQIndex,KOECGCSYIndex,KOECSGVQIndex,KOECSEMQIndex,KOECSIMQIndex,KOECFCOYIndex,KOECFCSYIndex,OEKRN022Index,KOGNICNYIndex,KOGNICUYIndex,KOGCGDPYIndex,KOGCSTOQIndex,KODFTOTYIndex,KOCPIIndex,KOCPIYOYIndex,KOCPIMOMIndex,SKCITTLIndex,SKCIYOYIndex,SKCIMOMIndex,KOCLLNSIndex,KOPPIIndex,KOPPIYOYIndex,KOPPIMOMIndex,KOEAUERSIndex,KOEAEMPYIndex,KOUETTYIndex,KOEAPYIndex,KOEAEPRIndex,KOQEEYOYIndex,KOWDRIndex,KOULMGFIndex,KOULMGFYIndex,KOLPIIndex,KOLPIYIndex,SKLIOTARIndex,KOIPISAIndex,KOIPIYIndex,KOIPIMOMIndex,KOIPMSIndex,KOIPMYIndex,KOIPMSMIndex,KOPSIYIndex,KOPIIYIndex,KOIPMCIndex,KOIPMCYIndex,KOIPOPSMIndex,KOIVCPEYIndex,KOIVCCYIndex,KOIVCONYIndex,KOCGCGYIndex,KOCGCGSMIndex,KOSVTOTYIndex,KOSVTOTMIndex,KOVMPRODIndex,KOBSMCIndex,KOBSNMCIndex,SKBSICIndex,SKBSICSAIndex,MPMIKRMAIndex,OEKRKLARIndex,OEKRKLAPIndex,SKLILIIndex,SKLILYIndex,SKLILCIndex,SKLICIIndex,SKLIINVCIndex,SKLICONEIndex,SKLIMORDIndex,SKLICONTIndex,SKLINBARIndex,SKLISPRCIndex,SKLIINTRIndex,SKLIIPALIndex,SKLICNSTIndex,SKLISVCIIndex,SKLIWNRSIndex,SKLISHPPIndex,SKLIIMPTIndex,SKLIEMPPIndex,SKLILAPIndex,SKLILAIIndex,SKLINEIndex,SKLICPYIndex,KOBONTLIndex,KOHSTRIndex,KOHPTTLIndex,KOHPTYOYIndex,KOHCTTLIndex,KOHCTYOYIndex,KOBLTHMLIndex,KOBLTHMYIndex,KOUHTTLIndex,KORSTIndex,KORSTYIndex,KODSDEPTIndex,KODSDISCIndex,KOVMTOTSIndex,KOVMDOMSIndex,KOVMEXPSIndex,KOCCCSIIndex,KOHOINCIndex,KOHOINCYIndex,KOHEUINYIndex,KOHHDIndex,KOHHDYIndex,KOHHLIndex,KOHHLYIndex,EOKOS002Index,KOBPCBIndex,EHCAKRYIndex,KOBPFINIndex,KOBPCAIndex,KODIBALIndex,KOBPTBIndex,KOTRBALIndex,KOEXTOTIndex,KOIMTOTIndex,KOEXTOTYIndex,KOIMTOTYIndex,KOEXPTIIndex,KOEXPTIYIndex,KOEXPTIMIndex,KOIMPTIIndex,KOIMPTIYIndex,KOIMPTIMIndex,KOFDITIndex,KOFDITYIndex,KOELTOTIndex,KOGFBALIndex,KOGFTEXPIndex,KOGFTRIndex,G7BBKOREIndex,GDDBSOKOIndex,KOMSMBYIndex,KOMSM1YIndex,KOMSM2YIndex,KOMSM3YIndex,KOMSLYIndex,KOMSM1FYIndex,KOFETOTIndex,KOFETOT%Index,KORP7DIndex,KOCICOBAIndex,KOCICONFIndex,KRBO3MIndex,KWCDCCurncy,GVSK10YRIndex,Amazon_close,Amazon_volume,Amazon_change,AMD_close,AMD_volume,AMD_change,APPLE_close,APPLE_volume,APPLE_change,AppliedMaterials_close,AppliedMaterials_vo

In [6]:
df = df.loc[9132 :, :]

In [7]:
df.reset_index(drop = True, inplace = True)

In [8]:
# 제대로 Column 들이 박혀있는 Dataframe이 들어오는 클래스
class Builder:

    def __init__(self, input_df, map_df):
        self.df = input_df
        self.mdf = map_df
        self.lag_days = [1, 7, 30, 90, 180, 365] # lagging day variation
        self.target_day = 1


    # df should involve datetime index

    def targeter(self, target_column):
        self.df[target_column] = self.df[target_column].shift(-1*int(self.target_day))

        return self.df

    def returner(self, target_column):      

        col_name =  []
        # creating new column name
        for i in self.lag_days:
            col_name.append(target_column + str(i))
        # appending lagged columns
        for i, j in enumerate(self.lag_days):
            self.df[locals()['col_name'][i]] = self.df[target_column] / self.df[target_column].shift(periods = j, axis = 0)

        return self.df 


    
    def differ(self, target_column):

        col_name = []
        for i in self.lag_days:
            col_name.append(target_column + str(i))
        for i, j in enumerate(self.lag_days):
            self.df[locals()['col_name'][i]] = self.df[target_column] - self.df[target_column].shift(periods = j, axis = 0)
        return self.df

    # volume * close  -> convert to return 
    # 안타깝게도 mdf 에 있는 이름은 '주가'_close 의 형태가 되어야 함
    def producter(self, target_column):

        col_name = (str(target_column).split('_'))[0]# j가 들어오면 이걸 받고         
        # 해당 함수 내부에 col_name을 넣으면 해당 j가 들어가서 product와 결합된 column 생성

        self.df[locals()['col_name'] + '_product'] = self.df[col_name + '_volume'] * self.df[col_name + '_close'] # j volume과 j close를 곱해서 

        return self.df
    
    
    def execution(self):
        for i, j in enumerate(self.mdf['ticker']): # 키 값들을 순차적으로 돌리면서 순회, 예외사항이 있는지? 아 else를 걸어두면 상관없겠구나. 딱 mdf에 있는 만큼만 반영될듯
            #if self.mdf.loc[i, :][0] in self.df.columns:
            if j in self.df.columns:

                if self.mdf.loc[i, :][1] == 'product': 
                    self.producter(j)
                    col = str(j).split('_')[0]
                    self.df[col + '_product'] = self.df[col + '_product'] / self.df[col + '_product'].shift(1)


                if self.mdf.loc[i, :][1] == 'target': 
                    self.targeter(j)
                    
                    # 여기에서 필요한 것만 남겨야 하는데



                else:
                    pass
            else:
                pass
                
                '''

                if self.mdf.loc[i, :][1] == 'minus':
                    self.differ(j)

                elif self.mdf.loc[i, :][1] == 'change':
                    self.returner(j)

                elif self.mdf.loc[i, :][1] == 'product': #왜 안돌까. Amazon은 되는데. 어딘선가 도는게 멈추는게 분명한데. self.mdf.ticker에서 
                    self.producter(j)
                    # 여기에 product를 return으로 만드는 code 추가

                '''
                #else:
                   # pass
           # else:
            #    pass
            
        return self.df


            
    # 차분에 ACF 그래프도 그리면 좋을듯. 정상성을 나타내지 않는 시계열에서는 ACF가 느리게 감소하고
    # 정상성을 나타내는 시계열에서는 ACF가 비교적 빠르게 0으로 떨어질 것
    # 구글 주식 가격 데이터의 경우 차분을 하면 ACF는 단순히 white noise time series 처럼 생겼음


    # spline이 된 상태에서는 acf를 적용할 수 있다. 
    # cubic spline만 적용한다고 가정하고
    # raw에서 채우지는 않고 acf만 보는 function을 짜자
    def acf_cal(self, target):
        data = pd.DataFrame(self.df[target])
        data = data.interpolate(method = 'cubic', limit_area = 'inside').fillna(method = 'ffill').fillna(method = 'bfill')
        acf = sm.tsa.acf(data)      
        plt.stem(acf)
        plt.show()

In [9]:
newdf = df.copy()

In [10]:
len(newdf.columns)

417

In [11]:
newdf['date'] = pd.to_datetime(newdf['date'])

In [12]:
newdf.set_index('date', inplace = True)

In [13]:
newdf = newdf.interpolate(method = 'cubic', limit_area = 'inside')
newdf = newdf.fillna(method = 'bfill')
newdf = newdf.fillna(method = 'ffill')

In [14]:
len(newdf.columns)

416

In [15]:
newdf.isna().sum().sum()

0

In [16]:
cla = Builder(newdf, mmdf)

In [17]:
cla.execution()

,GDPCQOQIndex,GDPCYOYIndex,GDPCTOT%Index,GPDITOC%Index,GPGSTOC%Index,RGCDCIPIIndex,GDPCUR$Index,GDPCURYIndex,GDPPIQQIndex,GDPCPCECIndex,CPIYOYIndex,CPIXYOYIndex,PCEDEFYIndex,PCECYOYIndex,CPICHNGIndex,CPUPXCHGIndex,FDIDFDMOIndex,FDIDSGMOIndex,FDIUFDYOIndex,FDIUSGYOIndex,USURTOTIndex,USUDMAERIndex,INJCJCIndex,INJCSPIndex,NFPTCHIndex,NFPPCHIndex,USMMMNCHIndex,AWHTOTLIndex,USWHTOTIndex,USEMTOTIndex,USEMNCHGIndex,USERTOTIndex,EMDINP1MIndex,JOLTOPENIndex,JOLTHIRSIndex,JOLTSEPSIndex,CHALYOY%Index,ADPCHNGIndex,USHEYOYIndex,USHETOT%Index,ECIYOYIndex,COSTNFR%Index,PRODNFR%Index,IPYOYIndex,IPCHNGIndex,CPTICHNGIndex,TMNOCHNGIndex,DGNOYOYIndex,DGNOCHNGIndex,MTIBCHNGIndex,MWINCHNGIndex,MGT2TBIndex,MGT2MAIndex,MGT2WHIndex,MGT2WHDUIndex,MGT2REIndex,IPIndex,MTSLRL$Index,AGWHTOTLIndex,NFPTIndex,PIDSPINXIndex,NAPMPMIIndex,NAPMNMIIndex,NAPMPRICIndex,MPMIUSCAIndex,MPMIUSMAIndex,MPMIUSSAIndex,CFNAIIndex,CFNAIMA3Index,OUTFGAFIndex,EMPRGBCIIndex,RCHSINDXIndex,CHPMINDXIndex,MAPMINDXIndex,OEUSKLARIndex,OEUSKLAPIndex,LEITOTLIndex,LEIYOYIndex,LEICHNGIndex,LEIAVGWIndex,LEIWKIJIndex,LEINWCNIndex,NAPMNEWOIndex,LEIMNOIndex,LEIBPIndex,LEISTKPIndex,LEILCIIndex,LEIIRTEIndex,LEIACEIndex,RSTAMOMIndex,RSTAXMOMIndex,RSTAYOYIndex,RSTAXYOYIndex,MBRXYOYWIndex,SAARTOTLIndex,SAARDTOTIndex,CONCCONFIndex,CONSSENTIndex,COMFCOMFIndex,COMFBTWRIndex,PITLYOYIndex,PITLCHNGIndex,PCECHY%Index,PCECHNCIndex,PIDSDPSIndex,CICRTOTIndex,EHCAUSIndex,USCABALIndex,USTBTOTIndex,USTBEXPIndex,USTBIMPIndex,USTBEXPYIndex,USTBIMPYIndex,IMP1YOY%Index,IMP1CHNGIndex,FRNTTOTLIndex,FRNTTNETIndex,FDTRIndex,USGG3MIndex,USGG10YRIndex,USTW$Index,JNVNYOYSIndex,JNVNIYOYIndex,MPMIJPCAIndex,MPMIJPMAIndex,MPMIJPSAIndex,CNGDPYOYIndex,GDPNTTLYIndex,CHEFTYOYIndex,CNPPIYIndex,CPMINDXIndex,CPMINORDIndex,CPMINEXOIndex,CNCILIIndex,CNFREXPYIndex,CNFRIMPYIndex,KOGDPYOYIndex,KOGDPQOQIndex,KOECPRCYIndex,KOECGCPYIndex,KOECGVTYIndex,KOECEXPYIndex,KOECIMPYIndex,KOECSPRQIndex,KOECGCSYIndex,KOECSGVQIndex,KOECSEMQIndex,KOECSIMQIndex,KOECFCOYIndex,KOECFCSYIndex,OEKRN022Index,KOGNICNYIndex,KOGNICUYIndex,KOGCGDPYIndex,KOGCSTOQIndex,KODFTOTYIndex,KOCPIIndex,KOCPIYOYIndex,KOCPIMOMIndex,SKCITTLIndex,SKCIYOYIndex,SKCIMOMIndex,KOCLLNSIndex,KOPPIIndex,KOPPIYOYIndex,KOPPIMOMIndex,KOEAUERSIndex,KOEAEMPYIndex,KOUETTYIndex,KOEAPYIndex,KOEAEPRIndex,KOQEEYOYIndex,KOWDRIndex,KOULMGFIndex,KOULMGFYIndex,KOLPIIndex,KOLPIYIndex,SKLIOTARIndex,KOIPISAIndex,KOIPIYIndex,KOIPIMOMIndex,KOIPMSIndex,KOIPMYIndex,KOIPMSMIndex,KOPSIYIndex,KOPIIYIndex,KOIPMCIndex,KOIPMCYIndex,KOIPOPSMIndex,KOIVCPEYIndex,KOIVCCYIndex,KOIVCONYIndex,KOCGCGYIndex,KOCGCGSMIndex,KOSVTOTYIndex,KOSVTOTMIndex,KOVMPRODIndex,KOBSMCIndex,KOBSNMCIndex,SKBSICIndex,SKBSICSAIndex,MPMIKRMAIndex,OEKRKLARIndex,OEKRKLAPIndex,SKLILIIndex,SKLILYIndex,SKLILCIndex,SKLICIIndex,SKLIINVCIndex,SKLICONEIndex,SKLIMORDIndex,SKLICONTIndex,SKLINBARIndex,SKLISPRCIndex,SKLIINTRIndex,SKLIIPALIndex,SKLICNSTIndex,SKLISVCIIndex,SKLIWNRSIndex,SKLISHPPIndex,SKLIIMPTIndex,SKLIEMPPIndex,SKLILAPIndex,SKLILAIIndex,SKLINEIndex,SKLICPYIndex,KOBONTLIndex,KOHSTRIndex,KOHPTTLIndex,KOHPTYOYIndex,KOHCTTLIndex,KOHCTYOYIndex,KOBLTHMLIndex,KOBLTHMYIndex,KOUHTTLIndex,KORSTIndex,KORSTYIndex,KODSDEPTIndex,KODSDISCIndex,KOVMTOTSIndex,KOVMDOMSIndex,KOVMEXPSIndex,KOCCCSIIndex,KOHOINCIndex,KOHOINCYIndex,KOHEUINYIndex,KOHHDIndex,KOHHDYIndex,KOHHLIndex,KOHHLYIndex,EOKOS002Index,KOBPCBIndex,EHCAKRYIndex,KOBPFINIndex,KOBPCAIndex,KODIBALIndex,KOBPTBIndex,KOTRBALIndex,KOEXTOTIndex,KOIMTOTIndex,KOEXTOTYIndex,KOIMTOTYIndex,KOEXPTIIndex,KOEXPTIYIndex,KOEXPTIMIndex,KOIMPTIIndex,KOIMPTIYIndex,KOIMPTIMIndex,KOFDITIndex,KOFDITYIndex,KOELTOTIndex,KOGFBALIndex,KOGFTEXPIndex,KOGFTRIndex,G7BBKOREIndex,GDDBSOKOIndex,KOMSMBYIndex,KOMSM1YIndex,KOMSM2YIndex,KOMSM3YIndex,KOMSLYIndex,KOMSM1FYIndex,KOFETOTIndex,KOFETOT%Index,KORP7DIndex,KOCICOBAIndex,KOCICONFIndex,KRBO3MIndex,KWCDCCurncy,GVSK10YRIndex,Amazon_close,Amazon_volume,Amazon_change,AMD_close,AMD_volume,AMD_change,APPLE_close,APPLE_volume,APPLE_change,AppliedMaterials_close,AppliedMaterials_volume,

In [18]:
newdf = cla.df.fillna(method = 'ffill').fillna(method = 'bfill')

In [19]:
newdf.isna().sum().sum()

0

In [20]:
len(newdf.columns)

442

In [21]:
newdf.isna().sum().sum()

0

# Scaling

In [22]:
newdf.reset_index(inplace = True)

In [23]:
date = newdf.date

In [24]:
newdf.drop('date', axis = True, inplace = True)

In [25]:
y_target = newdf['KRXsemiconductor_change']

In [26]:
# hynix, krx
newdf.drop(['KRXsemiconductor_close',
 'KRXsemiconductor_volume',
 'KRXsemiconductor_change'], axis = 1, inplace = True)

In [27]:
# scaling 하기전에 columns 뽑기 
col_name = newdf.columns

In [28]:
newdf.describe()

,GDPCQOQIndex,GDPCYOYIndex,GDPCTOT%Index,GPDITOC%Index,GPGSTOC%Index,RGCDCIPIIndex,GDPCUR$Index,GDPCURYIndex,GDPPIQQIndex,GDPCPCECIndex,CPIYOYIndex,CPIXYOYIndex,PCEDEFYIndex,PCECYOYIndex,CPICHNGIndex,CPUPXCHGIndex,FDIDFDMOIndex,FDIDSGMOIndex,FDIUFDYOIndex,FDIUSGYOIndex,USURTOTIndex,USUDMAERIndex,INJCJCIndex,INJCSPIndex,NFPTCHIndex,NFPPCHIndex,USMMMNCHIndex,AWHTOTLIndex,USWHTOTIndex,USEMTOTIndex,USEMNCHGIndex,USERTOTIndex,EMDINP1MIndex,JOLTOPENIndex,JOLTHIRSIndex,JOLTSEPSIndex,CHALYOY%Index,ADPCHNGIndex,USHEYOYIndex,USHETOT%Index,ECIYOYIndex,COSTNFR%Index,PRODNFR%Index,IPYOYIndex,IPCHNGIndex,CPTICHNGIndex,TMNOCHNGIndex,DGNOYOYIndex,DGNOCHNGIndex,MTIBCHNGIndex,MWINCHNGIndex,MGT2TBIndex,MGT2MAIndex,MGT2WHIndex,MGT2WHDUIndex,MGT2REIndex,IPIndex,MTSLRL$Index,AGWHTOTLIndex,NFPTIndex,PIDSPINXIndex,NAPMPMIIndex,NAPMNMIIndex,NAPMPRICIndex,MPMIUSCAIndex,MPMIUSMAIndex,MPMIUSSAIndex,CFNAIIndex,CFNAIMA3Index,OUTFGAFIndex,EMPRGBCIIndex,RCHSINDXIndex,CHPMINDXIndex,MAPMINDXIndex,OEUSKLARIndex,OEUSKLAPIndex,LEITOTLIndex,LEIYOYIndex,LEICHNGIndex,LEIAVGWIndex,LEIWKIJIndex,LEINWCNIndex,NAPMNEWOIndex,LEIMNOIndex,LEIBPIndex,LEISTKPIndex,LEILCIIndex,LEIIRTEIndex,LEIACEIndex,RSTAMOMIndex,RSTAXMOMIndex,RSTAYOYIndex,RSTAXYOYIndex,MBRXYOYWIndex,SAARTOTLIndex,SAARDTOTIndex,CONCCONFIndex,CONSSENTIndex,COMFCOMFIndex,COMFBTWRIndex,PITLYOYIndex,PITLCHNGIndex,PCECHY%Index,PCECHNCIndex,PIDSDPSIndex,CICRTOTIndex,EHCAUSIndex,USCABALIndex,USTBTOTIndex,USTBEXPIndex,USTBIMPIndex,USTBEXPYIndex,USTBIMPYIndex,IMP1YOY%Index,IMP1CHNGIndex,FRNTTOTLIndex,FRNTTNETIndex,FDTRIndex,USGG3MIndex,USGG10YRIndex,USTW$Index,JNVNYOYSIndex,JNVNIYOYIndex,MPMIJPCAIndex,MPMIJPMAIndex,MPMIJPSAIndex,CNGDPYOYIndex,GDPNTTLYIndex,CHEFTYOYIndex,CNPPIYIndex,CPMINDXIndex,CPMINORDIndex,CPMINEXOIndex,CNCILIIndex,CNFREXPYIndex,CNFRIMPYIndex,KOGDPYOYIndex,KOGDPQOQIndex,KOECPRCYIndex,KOECGCPYIndex,KOECGVTYIndex,KOECEXPYIndex,KOECIMPYIndex,KOECSPRQIndex,KOECGCSYIndex,KOECSGVQIndex,KOECSEMQIndex,KOECSIMQIndex,KOECFCOYIndex,KOECFCSYIndex,OEKRN022Index,KOGNICNYIndex,KOGNICUYIndex,KOGCGDPYIndex,KOGCSTOQIndex,KODFTOTYIndex,KOCPIIndex,KOCPIYOYIndex,KOCPIMOMIndex,SKCITTLIndex,SKCIYOYIndex,SKCIMOMIndex,KOCLLNSIndex,KOPPIIndex,KOPPIYOYIndex,KOPPIMOMIndex,KOEAUERSIndex,KOEAEMPYIndex,KOUETTYIndex,KOEAPYIndex,KOEAEPRIndex,KOQEEYOYIndex,KOWDRIndex,KOULMGFIndex,KOULMGFYIndex,KOLPIIndex,KOLPIYIndex,SKLIOTARIndex,KOIPISAIndex,KOIPIYIndex,KOIPIMOMIndex,KOIPMSIndex,KOIPMYIndex,KOIPMSMIndex,KOPSIYIndex,KOPIIYIndex,KOIPMCIndex,KOIPMCYIndex,KOIPOPSMIndex,KOIVCPEYIndex,KOIVCCYIndex,KOIVCONYIndex,KOCGCGYIndex,KOCGCGSMIndex,KOSVTOTYIndex,KOSVTOTMIndex,KOVMPRODIndex,KOBSMCIndex,KOBSNMCIndex,SKBSICIndex,SKBSICSAIndex,MPMIKRMAIndex,OEKRKLARIndex,OEKRKLAPIndex,SKLILIIndex,SKLILYIndex,SKLILCIndex,SKLICIIndex,SKLIINVCIndex,SKLICONEIndex,SKLIMORDIndex,SKLICONTIndex,SKLINBARIndex,SKLISPRCIndex,SKLIINTRIndex,SKLIIPALIndex,SKLICNSTIndex,SKLISVCIIndex,SKLIWNRSIndex,SKLISHPPIndex,SKLIIMPTIndex,SKLIEMPPIndex,SKLILAPIndex,SKLILAIIndex,SKLINEIndex,SKLICPYIndex,KOBONTLIndex,KOHSTRIndex,KOHPTTLIndex,KOHPTYOYIndex,KOHCTTLIndex,KOHCTYOYIndex,KOBLTHMLIndex,KOBLTHMYIndex,KOUHTTLIndex,KORSTIndex,KORSTYIndex,KODSDEPTIndex,KODSDISCIndex,KOVMTOTSIndex,KOVMDOMSIndex,KOVMEXPSIndex,KOCCCSIIndex,KOHOINCIndex,KOHOINCYIndex,KOHEUINYIndex,KOHHDIndex,KOHHDYIndex,KOHHLIndex,KOHHLYIndex,EOKOS002Index,KOBPCBIndex,EHCAKRYIndex,KOBPFINIndex,KOBPCAIndex,KODIBALIndex,KOBPTBIndex,KOTRBALIndex,KOEXTOTIndex,KOIMTOTIndex,KOEXTOTYIndex,KOIMTOTYIndex,KOEXPTIIndex,KOEXPTIYIndex,KOEXPTIMIndex,KOIMPTIIndex,KOIMPTIYIndex,KOIMPTIMIndex,KOFDITIndex,KOFDITYIndex,KOELTOTIndex,KOGFBALIndex,KOGFTEXPIndex,KOGFTRIndex,G7BBKOREIndex,GDDBSOKOIndex,KOMSMBYIndex,KOMSM1YIndex,KOMSM2YIndex,KOMSM3YIndex,KOMSLYIndex,KOMSM1FYIndex,KOFETOTIndex,KOFETOT%Index,KORP7DIndex,KOCICOBAIndex,KOCICONFIndex,KRBO3MIndex,KWCDCCurncy,GVSK10YRIndex,Amazon_close,Amazon_volume,Amazon_change,AMD_close,AMD_volume,AMD_change,APPLE_close,APPLE_volume,APPLE_change,AppliedMaterials_close,AppliedMaterials_volume,

In [29]:
# 삼성전자의 데이터가 infinity를 포함하고 있는 오류 발견

In [30]:
newdf.drop('SamsungElectronics_product', inplace = True, axis = 1) # 왜 삼성전자가 infinity가 뜰까??

In [31]:
# scaling과 동시에 변환
#newdf = PowerTransformer().fit_transform(newdf)
newdf = StandardScaler().fit_transform(newdf)
#newdf = newdf.apply(lambda x : np.log1p(x))
#df = MinMaxScaler().fit_transfrom(df)

In [32]:
newdf

array([[ 1.59613249,  1.00292133,  1.40363953, ..., -0.02695322,
        -0.07412472, -0.02519408],
       [ 1.59613249,  1.00292133,  1.40363953, ..., -0.02695322,
        -0.07412472, -0.02519408],
       [ 1.59613249,  1.00292133,  1.40363953, ..., -0.02695322,
        -0.23125665, -0.06978741],
       ...,
       [ 0.04909595,  0.13054771,  0.49456347, ..., -0.01385885,
         0.56856346,  0.13702959],
       [ 0.04909595,  0.13054771,  0.49456347, ...,  0.03757343,
        -0.16525095, -0.09224981],
       [ 0.04909595,  0.13054771,  0.49456347, ...,  0.01825286,
         0.16741648,  0.15127812]])

In [33]:
col_name=col_name.drop('SamsungElectronics_product')

In [34]:
newdf = pd.DataFrame(newdf, columns = col_name)

In [35]:
newdf.describe()

,GDPCQOQIndex,GDPCYOYIndex,GDPCTOT%Index,GPDITOC%Index,GPGSTOC%Index,RGCDCIPIIndex,GDPCUR$Index,GDPCURYIndex,GDPPIQQIndex,GDPCPCECIndex,CPIYOYIndex,CPIXYOYIndex,PCEDEFYIndex,PCECYOYIndex,CPICHNGIndex,CPUPXCHGIndex,FDIDFDMOIndex,FDIDSGMOIndex,FDIUFDYOIndex,FDIUSGYOIndex,USURTOTIndex,USUDMAERIndex,INJCJCIndex,INJCSPIndex,NFPTCHIndex,NFPPCHIndex,USMMMNCHIndex,AWHTOTLIndex,USWHTOTIndex,USEMTOTIndex,USEMNCHGIndex,USERTOTIndex,EMDINP1MIndex,JOLTOPENIndex,JOLTHIRSIndex,JOLTSEPSIndex,CHALYOY%Index,ADPCHNGIndex,USHEYOYIndex,USHETOT%Index,ECIYOYIndex,COSTNFR%Index,PRODNFR%Index,IPYOYIndex,IPCHNGIndex,CPTICHNGIndex,TMNOCHNGIndex,DGNOYOYIndex,DGNOCHNGIndex,MTIBCHNGIndex,MWINCHNGIndex,MGT2TBIndex,MGT2MAIndex,MGT2WHIndex,MGT2WHDUIndex,MGT2REIndex,IPIndex,MTSLRL$Index,AGWHTOTLIndex,NFPTIndex,PIDSPINXIndex,NAPMPMIIndex,NAPMNMIIndex,NAPMPRICIndex,MPMIUSCAIndex,MPMIUSMAIndex,MPMIUSSAIndex,CFNAIIndex,CFNAIMA3Index,OUTFGAFIndex,EMPRGBCIIndex,RCHSINDXIndex,CHPMINDXIndex,MAPMINDXIndex,OEUSKLARIndex,OEUSKLAPIndex,LEITOTLIndex,LEIYOYIndex,LEICHNGIndex,LEIAVGWIndex,LEIWKIJIndex,LEINWCNIndex,NAPMNEWOIndex,LEIMNOIndex,LEIBPIndex,LEISTKPIndex,LEILCIIndex,LEIIRTEIndex,LEIACEIndex,RSTAMOMIndex,RSTAXMOMIndex,RSTAYOYIndex,RSTAXYOYIndex,MBRXYOYWIndex,SAARTOTLIndex,SAARDTOTIndex,CONCCONFIndex,CONSSENTIndex,COMFCOMFIndex,COMFBTWRIndex,PITLYOYIndex,PITLCHNGIndex,PCECHY%Index,PCECHNCIndex,PIDSDPSIndex,CICRTOTIndex,EHCAUSIndex,USCABALIndex,USTBTOTIndex,USTBEXPIndex,USTBIMPIndex,USTBEXPYIndex,USTBIMPYIndex,IMP1YOY%Index,IMP1CHNGIndex,FRNTTOTLIndex,FRNTTNETIndex,FDTRIndex,USGG3MIndex,USGG10YRIndex,USTW$Index,JNVNYOYSIndex,JNVNIYOYIndex,MPMIJPCAIndex,MPMIJPMAIndex,MPMIJPSAIndex,CNGDPYOYIndex,GDPNTTLYIndex,CHEFTYOYIndex,CNPPIYIndex,CPMINDXIndex,CPMINORDIndex,CPMINEXOIndex,CNCILIIndex,CNFREXPYIndex,CNFRIMPYIndex,KOGDPYOYIndex,KOGDPQOQIndex,KOECPRCYIndex,KOECGCPYIndex,KOECGVTYIndex,KOECEXPYIndex,KOECIMPYIndex,KOECSPRQIndex,KOECGCSYIndex,KOECSGVQIndex,KOECSEMQIndex,KOECSIMQIndex,KOECFCOYIndex,KOECFCSYIndex,OEKRN022Index,KOGNICNYIndex,KOGNICUYIndex,KOGCGDPYIndex,KOGCSTOQIndex,KODFTOTYIndex,KOCPIIndex,KOCPIYOYIndex,KOCPIMOMIndex,SKCITTLIndex,SKCIYOYIndex,SKCIMOMIndex,KOCLLNSIndex,KOPPIIndex,KOPPIYOYIndex,KOPPIMOMIndex,KOEAUERSIndex,KOEAEMPYIndex,KOUETTYIndex,KOEAPYIndex,KOEAEPRIndex,KOQEEYOYIndex,KOWDRIndex,KOULMGFIndex,KOULMGFYIndex,KOLPIIndex,KOLPIYIndex,SKLIOTARIndex,KOIPISAIndex,KOIPIYIndex,KOIPIMOMIndex,KOIPMSIndex,KOIPMYIndex,KOIPMSMIndex,KOPSIYIndex,KOPIIYIndex,KOIPMCIndex,KOIPMCYIndex,KOIPOPSMIndex,KOIVCPEYIndex,KOIVCCYIndex,KOIVCONYIndex,KOCGCGYIndex,KOCGCGSMIndex,KOSVTOTYIndex,KOSVTOTMIndex,KOVMPRODIndex,KOBSMCIndex,KOBSNMCIndex,SKBSICIndex,SKBSICSAIndex,MPMIKRMAIndex,OEKRKLARIndex,OEKRKLAPIndex,SKLILIIndex,SKLILYIndex,SKLILCIndex,SKLICIIndex,SKLIINVCIndex,SKLICONEIndex,SKLIMORDIndex,SKLICONTIndex,SKLINBARIndex,SKLISPRCIndex,SKLIINTRIndex,SKLIIPALIndex,SKLICNSTIndex,SKLISVCIIndex,SKLIWNRSIndex,SKLISHPPIndex,SKLIIMPTIndex,SKLIEMPPIndex,SKLILAPIndex,SKLILAIIndex,SKLINEIndex,SKLICPYIndex,KOBONTLIndex,KOHSTRIndex,KOHPTTLIndex,KOHPTYOYIndex,KOHCTTLIndex,KOHCTYOYIndex,KOBLTHMLIndex,KOBLTHMYIndex,KOUHTTLIndex,KORSTIndex,KORSTYIndex,KODSDEPTIndex,KODSDISCIndex,KOVMTOTSIndex,KOVMDOMSIndex,KOVMEXPSIndex,KOCCCSIIndex,KOHOINCIndex,KOHOINCYIndex,KOHEUINYIndex,KOHHDIndex,KOHHDYIndex,KOHHLIndex,KOHHLYIndex,EOKOS002Index,KOBPCBIndex,EHCAKRYIndex,KOBPFINIndex,KOBPCAIndex,KODIBALIndex,KOBPTBIndex,KOTRBALIndex,KOEXTOTIndex,KOIMTOTIndex,KOEXTOTYIndex,KOIMTOTYIndex,KOEXPTIIndex,KOEXPTIYIndex,KOEXPTIMIndex,KOIMPTIIndex,KOIMPTIYIndex,KOIMPTIMIndex,KOFDITIndex,KOFDITYIndex,KOELTOTIndex,KOGFBALIndex,KOGFTEXPIndex,KOGFTRIndex,G7BBKOREIndex,GDDBSOKOIndex,KOMSMBYIndex,KOMSM1YIndex,KOMSM2YIndex,KOMSM3YIndex,KOMSLYIndex,KOMSM1FYIndex,KOFETOTIndex,KOFETOT%Index,KORP7DIndex,KOCICOBAIndex,KOCICONFIndex,KRBO3MIndex,KWCDCCurncy,GVSK10YRIndex,Amazon_close,Amazon_volume,Amazon_change,AMD_close,AMD_volume,AMD_change,APPLE_close,APPLE_volume,APPLE_change,AppliedMaterials_close,AppliedMaterials_volume,

In [36]:
len(newdf.columns)

438

In [158]:
for i in newdf.columns:
    if newdf[i].std() == 0:
        newdf.drop(i, axis = 1, inplace = True)

In [38]:
newdf.describe()

,GDPCQOQIndex,GDPCYOYIndex,GDPCTOT%Index,GPDITOC%Index,GPGSTOC%Index,RGCDCIPIIndex,GDPCUR$Index,GDPCURYIndex,GDPPIQQIndex,GDPCPCECIndex,CPIYOYIndex,CPIXYOYIndex,PCEDEFYIndex,PCECYOYIndex,CPICHNGIndex,CPUPXCHGIndex,FDIDFDMOIndex,FDIDSGMOIndex,FDIUFDYOIndex,FDIUSGYOIndex,USURTOTIndex,USUDMAERIndex,INJCJCIndex,INJCSPIndex,NFPTCHIndex,NFPPCHIndex,USMMMNCHIndex,AWHTOTLIndex,USWHTOTIndex,USEMTOTIndex,USEMNCHGIndex,USERTOTIndex,EMDINP1MIndex,JOLTOPENIndex,JOLTHIRSIndex,JOLTSEPSIndex,CHALYOY%Index,ADPCHNGIndex,USHEYOYIndex,USHETOT%Index,ECIYOYIndex,COSTNFR%Index,PRODNFR%Index,IPYOYIndex,IPCHNGIndex,CPTICHNGIndex,TMNOCHNGIndex,DGNOYOYIndex,DGNOCHNGIndex,MTIBCHNGIndex,MWINCHNGIndex,MGT2TBIndex,MGT2MAIndex,MGT2WHIndex,MGT2WHDUIndex,MGT2REIndex,IPIndex,MTSLRL$Index,AGWHTOTLIndex,NFPTIndex,PIDSPINXIndex,NAPMPMIIndex,NAPMNMIIndex,NAPMPRICIndex,MPMIUSCAIndex,MPMIUSMAIndex,MPMIUSSAIndex,CFNAIIndex,CFNAIMA3Index,OUTFGAFIndex,EMPRGBCIIndex,RCHSINDXIndex,CHPMINDXIndex,MAPMINDXIndex,OEUSKLARIndex,OEUSKLAPIndex,LEITOTLIndex,LEIYOYIndex,LEICHNGIndex,LEIAVGWIndex,LEIWKIJIndex,LEINWCNIndex,NAPMNEWOIndex,LEIMNOIndex,LEIBPIndex,LEISTKPIndex,LEILCIIndex,LEIIRTEIndex,LEIACEIndex,RSTAMOMIndex,RSTAXMOMIndex,RSTAYOYIndex,RSTAXYOYIndex,MBRXYOYWIndex,SAARTOTLIndex,SAARDTOTIndex,CONCCONFIndex,CONSSENTIndex,COMFCOMFIndex,COMFBTWRIndex,PITLYOYIndex,PITLCHNGIndex,PCECHY%Index,PCECHNCIndex,PIDSDPSIndex,CICRTOTIndex,EHCAUSIndex,USCABALIndex,USTBTOTIndex,USTBEXPIndex,USTBIMPIndex,USTBEXPYIndex,USTBIMPYIndex,IMP1YOY%Index,IMP1CHNGIndex,FRNTTOTLIndex,FRNTTNETIndex,FDTRIndex,USGG3MIndex,USGG10YRIndex,USTW$Index,JNVNYOYSIndex,JNVNIYOYIndex,MPMIJPCAIndex,MPMIJPMAIndex,MPMIJPSAIndex,CNGDPYOYIndex,GDPNTTLYIndex,CHEFTYOYIndex,CNPPIYIndex,CPMINDXIndex,CPMINORDIndex,CPMINEXOIndex,CNCILIIndex,CNFREXPYIndex,CNFRIMPYIndex,KOGDPYOYIndex,KOGDPQOQIndex,KOECPRCYIndex,KOECGCPYIndex,KOECGVTYIndex,KOECEXPYIndex,KOECIMPYIndex,KOECSPRQIndex,KOECGCSYIndex,KOECSGVQIndex,KOECSEMQIndex,KOECSIMQIndex,KOECFCOYIndex,KOECFCSYIndex,OEKRN022Index,KOGNICNYIndex,KOGNICUYIndex,KOGCGDPYIndex,KOGCSTOQIndex,KODFTOTYIndex,KOCPIIndex,KOCPIYOYIndex,KOCPIMOMIndex,SKCITTLIndex,SKCIYOYIndex,SKCIMOMIndex,KOCLLNSIndex,KOPPIIndex,KOPPIYOYIndex,KOPPIMOMIndex,KOEAUERSIndex,KOEAEMPYIndex,KOUETTYIndex,KOEAPYIndex,KOEAEPRIndex,KOQEEYOYIndex,KOWDRIndex,KOULMGFIndex,KOULMGFYIndex,KOLPIIndex,KOLPIYIndex,SKLIOTARIndex,KOIPISAIndex,KOIPIYIndex,KOIPIMOMIndex,KOIPMSIndex,KOIPMYIndex,KOIPMSMIndex,KOPSIYIndex,KOPIIYIndex,KOIPMCIndex,KOIPMCYIndex,KOIPOPSMIndex,KOIVCPEYIndex,KOIVCCYIndex,KOIVCONYIndex,KOCGCGYIndex,KOCGCGSMIndex,KOSVTOTYIndex,KOSVTOTMIndex,KOVMPRODIndex,KOBSMCIndex,KOBSNMCIndex,SKBSICIndex,SKBSICSAIndex,MPMIKRMAIndex,OEKRKLARIndex,OEKRKLAPIndex,SKLILIIndex,SKLILYIndex,SKLILCIndex,SKLICIIndex,SKLIINVCIndex,SKLICONEIndex,SKLIMORDIndex,SKLICONTIndex,SKLINBARIndex,SKLISPRCIndex,SKLIINTRIndex,SKLIIPALIndex,SKLICNSTIndex,SKLISVCIIndex,SKLIWNRSIndex,SKLISHPPIndex,SKLIIMPTIndex,SKLIEMPPIndex,SKLILAPIndex,SKLILAIIndex,SKLINEIndex,SKLICPYIndex,KOBONTLIndex,KOHSTRIndex,KOHPTTLIndex,KOHPTYOYIndex,KOHCTTLIndex,KOHCTYOYIndex,KOBLTHMLIndex,KOBLTHMYIndex,KOUHTTLIndex,KORSTIndex,KORSTYIndex,KODSDEPTIndex,KODSDISCIndex,KOVMTOTSIndex,KOVMDOMSIndex,KOVMEXPSIndex,KOCCCSIIndex,KOHOINCIndex,KOHOINCYIndex,KOHEUINYIndex,KOHHDIndex,KOHHDYIndex,KOHHLIndex,KOHHLYIndex,EOKOS002Index,KOBPCBIndex,EHCAKRYIndex,KOBPFINIndex,KOBPCAIndex,KODIBALIndex,KOBPTBIndex,KOTRBALIndex,KOEXTOTIndex,KOIMTOTIndex,KOEXTOTYIndex,KOIMTOTYIndex,KOEXPTIIndex,KOEXPTIYIndex,KOEXPTIMIndex,KOIMPTIIndex,KOIMPTIYIndex,KOIMPTIMIndex,KOFDITIndex,KOFDITYIndex,KOELTOTIndex,KOGFBALIndex,KOGFTEXPIndex,KOGFTRIndex,G7BBKOREIndex,GDDBSOKOIndex,KOMSMBYIndex,KOMSM1YIndex,KOMSM2YIndex,KOMSM3YIndex,KOMSLYIndex,KOMSM1FYIndex,KOFETOTIndex,KOFETOT%Index,KORP7DIndex,KOCICOBAIndex,KOCICONFIndex,KRBO3MIndex,KWCDCCurncy,GVSK10YRIndex,Amazon_close,Amazon_volume,Amazon_change,AMD_close,AMD_volume,AMD_change,APPLE_close,APPLE_volume,APPLE_change,AppliedMaterials_close,AppliedMaterials_volume,

In [39]:
len(newdf.columns)

438

In [37]:
newdf.isna().sum().sum()

0

In [42]:
# KRX
X_data = newdf.copy()

In [43]:
# random split과 not random split
#X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size = 0.3, random_state = 156)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size = 0.2, shuffle = False)

# Linear Model

In [60]:
# stepwise selection

np.random.seed(0)
feature_name = list(X_train.columns)

In [61]:
X_train.shape

(4040, 438)

In [62]:
len(feature_name)

438

In [71]:
feature_selected_final

['SKhynix_change',
 'Philadelphia_change',
 'Micron_change',
 'MediaTek_change',
 'USDollarIndex_close',
 'FRNTTNETIndex',
 'Intel_product',
 'USD_KRW_change',
 'CPUPXCHGIndex',
 'KOBONTLIndex',
 'RSTAMOMIndex',
 'KOEXPTIYIndex',
 'KOCPIMOMIndex',
 'Dell_close',
 'Dell_volume',
 'GPGSTOC%Index',
 'SKLIOTARIndex',
 'Nvidia_volume',
 'SKCIMOMIndex',
 'KOCGCGYIndex',
 'KOECPRCYIndex',
 'LEIBPIndex',
 'AMD_change',
 'JOLTSEPSIndex',
 'HPE_close',
 'Sumco_change',
 'KOGFBALIndex',
 'CNFREXPYIndex',
 'TokyoElectron_close',
 'EUR_KRW_close',
 'KOBPFINIndex',
 'Amazon_change',
 'UMC_product',
 'KOECGVTYIndex',
 'KOBPCAIndex',
 'KOECIMPYIndex',
 'NAPMNMIIndex',
 'USWHTOTIndex',
 'Amazon_volume',
 'KOECFCSYIndex',
 'Google_product',
 'KOFDITYIndex',
 'KOQEEYOYIndex',
 'TokyoElectron_volume',
 'CNGDPYOYIndex',
 'MBRXYOYWIndex',
 'ShinEtsuChemical_volume',
 'USTBEXPYIndex',
 'AppliedMaterials_product',
 'QCOM_product',
 'SKhynix_product',
 'UMC_volume',
 'APPLE_product',
 'KOECSIMQIndex',
 'SKLICP

In [78]:
score[np.array(score).argmax()]

0.27386104369500763

In [79]:
feature_selected = []
score = []

for i in range(X_train.shape[1]): # 438 개 columns
    score_temp_list = np.zeros(len(feature_name)) # 438개의 zeros
    for j in range(len(feature_name)):
        temp = feature_selected.copy() # temp에 복사 <- 그 전에 만들어진 feature_selected에서 갖고 옴 ## 그리고 다시 돌아와서 제일 높은 feature를 복사하고 또 순서대로 추가해 간다. 
        temp.append(feature_name[j]) # feature_name 전체 중에 j번째(0~438중) 를 현재 feature목록에 추가
        x_temp = X_train[temp] # 해당 column들만으로 df생성
        lr_temp = LinearRegression() 
        lr_temp.fit(x_temp, y_train) # 그 column들만으로 fit
        predict_temp = lr_temp.predict(X_test[temp]) # 그리고 예측하고
        #mse_tmep = mean_squared_error(y_test, predict_temp)
        #rmse_temp = np.sqrt(mse_temp)
        r2_temp = r2_score(y_test, predict_temp)
        score_temp = r2_temp#[rmse_temp, r2_temp] # 만들자 
        score_temp_list[j] = score_temp # score temp를 0~ 0 으로 이뤄진 score_temp_list의 j번째 위치에 박아 넣는다. 
        # 이렇게 한바퀴가 돌아서 각 feature column의 위치에 해당 column을 추가했을 때의 성적이 다 박힌다. 
    # 그렇게 만들어진 score temp list에서 제일 높은 성적을 보였던 column의 위치를 
    # feature name에 넣어서 해당 column을 보존하고
    # 그걸 feature_selected에 넣어서 보관한다. 
    # 그리고 해당 column을 전체 feature_name에서 지운다.
    # 그러면 나머지 중에서 다시 돌겠지. 
    # 해당 순서대로 score 도 해당 위치에 추가 된다. 
        
    temp = feature_name[score_temp_list.argmax()]
    feature_selected.append(temp)
    feature_name.remove(temp)
    score.append(score_temp_list.max())
    print("%02d Selected: "%i, feature_selected)
    print("%02d Score : "%i, np.round(10000*np.array(score))/10000)
        
# finally selected features
feature_selected_final = feature_selected[:(np.array(score).argmax() + 1)]
        

IndentationError: unexpected indent (<ipython-input-79-7075bd46d31f>, line 5)

In [58]:
len(f_sel)

438

In [113]:
lr = LinearRegression()
ridge = Ridge(alpha = 3000)
lasso = Lasso(alpha = 1000)

In [302]:
def get_linear_model_r2score(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    print("model name :", model.__class__.__name__)
    print('MSE: {0: .3f}, RMSE : {1:.3F}'.format(mse, rmse))
    print('Variance Score : {0:3f}'.format(r2_score(y_test, y_pred)))

In [114]:
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [115]:
y_pred = lr.predict(X_test)

In [303]:
# 트리 기반 회귀 모델을 반복하며 평가 수행
linear_models = [lr, ridge, lasso]
for model in linear_models:
    get_linear_model_r2score(model, X_train, y_train, X_test, y_test)

model name : LinearRegression
MSE:  55060223.005, RMSE : 7420.258
Variance Score : -15180686.538819
model name : Ridge
MSE:  2.899, RMSE : 1.703
Variance Score : 0.200657
model name : Lasso
MSE:  3.628, RMSE : 1.905
Variance Score : -0.000145


# Tree Model

In [175]:
dt_reg = DecisionTreeRegressor(random_state = 0, max_depth = 100)
#rf_reg = RandomForestRegressor(random_state = 0, n_estimators = 1000)
#gb_reg = GradientBoostingRegressor(random_state = 0, n_estimators = 1000)
#xgb_reg = XGBRegressor(n_estimators = 1000)
lgb_reg = LGBMRegressor(n_estimators = 3000)

In [176]:
def get_tree_model_r2score(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    print("model name :", model.__class__.__name__)
    print('MSE: {0: .3f}, RMSE : {1:.3F}'.format(mse, rmse))
    print('Variance Score : {0:3f}'.format(r2_score(y_test, y_pred)))

In [177]:
# 트리 기반 회귀 모델을 반복하며 평가 수행
tree_models = [dt_reg, lgb_reg]
for model in tree_models:
    get_tree_model_r2score(model, X_train, y_train, X_test, y_test)

model name : DecisionTreeRegressor
MSE:  6.149, RMSE : 2.480
Variance Score : -0.695303
model name : LGBMRegressor
MSE:  3.016, RMSE : 1.737
Variance Score : 0.168373


In [305]:
ridge.fit(X_train, y_train)

Ridge(alpha=3000, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [307]:
y_pred = ridge.predict(X_test)

In [44]:
xgb_reg = XGBRegressor(n_estimators = 1000, learning_rate =0.1, max_depth = 100)

In [45]:
xgb_reg.fit(X_train, y_train)

[15:26:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=100, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
lgb_reg = LGBMRegressor(n_estimators = 1000, learning_rate =0.1, max_depth = 100)

In [91]:
lgb_reg.fit(X_train, y_train)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=1000, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [221]:
feat_imp = pd.DataFrame(sorted(zip(lgb_reg.feature_importances_, X_train.columns)), columns = ['Value', 'Feature'])

In [1]:
feat_imp.sort_values(by = 'Value', ascending = False)[0]

NameError: name 'feat_imp' is not defined

In [48]:
plt.figure(figsize = (20, 200))
sns.barplot(x = 'Value', y = 'Feature', data = feat_imp.sort_values(by = 'Value', ascending = False))
plt.title('LightGBM Features')
plt.tight_layout()
plt.savefig('lgbm_importances_1.png')
plt.show()


NameError: name 'feat_imp' is not defined

In [ ]:
# threshold 발라내는 코드는 이렇게 구성하면 되겠다 .

def get_outlier(df = None, column = None, weight = 1.5):
    quantile_25 = np.percentile(values, 25)
    quantile_75 = np.percentile(values, 75)
    
    outlier_index = data[( data < quantile25) | (data > quantile75)].index
    return outlier_index

# df.drop(outlier_index, axis = 0, inplace = True)

In [109]:
y_pred

NameError: name 'y_pred' is not defined

In [153]:
    def issame(data1, data2):
        if data1 > 0 and data2 > 0 : 
            return 'T'
        elif data1 > 0 and data2 < 0 :
            return 'F'
        elif data1 < 0 and data2 > 0 :
            return 'F'
        elif data1 == 0 or data2 == 0 :
            return 0
        else :
            return 'T'
    
    def getouter(data1, data2):
        if data1 > thr and data2 > thr:
            return 'T'
        elif data1 < thr and data2 < thr:
            return 'T'
        elif data1 == 0 or data2 == 0 :
            return 0
        else:
            return 'F'

In [156]:
    thr = 0.3


In [258]:
# metric 구성

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, accuracy_score

class CustomMetric():
    
    thr = 0.3

    def issame(data1, data2):
        if data1 > 0 and data2 > 0 : 
            return 'T'
        elif data1 > 0 and data2 < 0 :
            return 'F'
        elif data1 < 0 and data2 > 0 :
            return 'F'
        elif data1 == 0 or data2 == 0 :
            return 0
        else :
            return 'T'
    
    def getouter(data1, data2):
        if data1 > thr and data2 > thr:
            return 'T'
        elif data1 < thr and data2 < thr:
            return 'T'
        elif data1 == 0 or data2 == 0 :
            return 0
        else:
            return 'F'

    def __init__(self, pred, real):
        self.df = pd.DataFrame({'pred' : pred, 'real' : real}) # series 형태로 2개가 들어와야 함
        self.df['TF'] = self.df.apply(lambda x : issame(x['pred'], x['real']), axis = 1)
        self.df['thrTF'] = self.df.apply(lambda x : getouter(x['pred'], x['real']), axis = 1)
        
        
    def execution(self):
        mdf = pd.DataFrame()
        mdf['CORR'] = [self.df['pred'].corr(self.df['real'], method = 'pearson')]
        mdf['R2'] = ["{0:0.4f}".format(r2_score(self.df['pred'], self.df['real']))]
        mdf['MAE'] = ["{0:0.4f}".format(mean_absolute_error(self.df['pred'], self.df['real']))]
        mdf['RMSE'] = ["{0:0.4f}".format(np.sqrt(mean_squared_error(self.df['pred'], self.df['real'])))]
        mdf['ACR'] = [sum(self.df['TF'] == 'T')/len(self.df['TF'])]
        mdf['threshACR'] = [sum(self.df['thrTF'] == 'T')/len(self.df['thrTF'])]
        
        return mdf        
    

In [259]:
metr = CustomMetric(y_pred, y_test)

In [260]:
metr.execution()

,CORR,R2,MAE,RMSE,ACR,threshACR
0,-0.102427,-1.6207,9473569898776.4961,11287369603152.7051,0.473788,0.4273


In [267]:
sum(metr.df['thrTF'] == 'T')

432

In [262]:
len(metr.df['thrTF'])

1011